<a href="https://colab.research.google.com/github/KARTIKAY2003/Project/blob/main/Babaji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 3s (1,808 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123599 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install pytesseract opencv-python scikit-learn pandas

In [ ]:
import pytesseract
import cv2
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Step 2: Upload Image (You can use any invoice image)
from google.colab import files
uploaded = files.upload()

Saving Screenshot 2024-09-20 at 5.00.37 PM.png to Screenshot 2024-09-20 at 5.00.37 PM.png


In [ ]:
# Step 3: Read and Preprocess the Image using OpenCV
image_path = list(uploaded.keys())[0]  # Get the uploaded file's name
image = cv2.imread("/content/Screenshot 2024-09-20 at 5.00.37 PM.png")

In [ ]:
# Convert the image to grayscale for better OCR performance
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# Optional: Apply thresholding to preprocess the image further
_, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

In [ ]:
# Step 4: Extract Text Using Tesseract OCR
text = pytesseract.image_to_string(thresh_image)
print("Extracted Text from Invoice:\n", text)

Extracted Text from Invoice:
 amazonin

Sold By :

CLICKTECH RETAIL PRIVATE LIMITED
“Khasra numbers:444(P),445(P),459(P),
460,461 462,463,464,
465,466,467,468,469,470,471,472,473,474,,,
75(P),476,477,478, 479,480,

481 ,482,483(P),491 ,492,493(P) Village -
Bhaukapur,

Lucknow, Uttar Pradesh, 226401

IN

PAN No:AAJCC9783E
GST Registration No:09AAJCC9783E1Z5
Dynamic QR Code:

 

Order Number:402-4170507-6103552
Order Date:23.08.2024

Tax Invoice/Bill of Supply/Cash Memo
(Original for Recipient)

Billing Address :

Pradeep Kumar

Kamaldeep House, Maruti Nagar
RAWATBHATA, RAJASTHAN, 323307
IN

State/UT Code:08

Shipping Address :

Pradeep Kumar

Pradeep Kumar

Kamaldeep House, Maruti Nagar
RAWATBHATA, RAJASTHAN, 323307
IN

State/UT Code:08

Place of supply: RAJASTHAN

Place of delivery: RAJASTHAN

Invoice Number :LKO1-721112
Invoice Details :UP-LKO1-297683823-2425
Invoice Date :23.08.2024

 

SI.

No| Description

Unit
Price

Net [Tax |Tax |Tax [Total
|Amount|Rate|Type|Amount/Amount

 

IB

In [ ]:
# Step 5: Detect the Bill Amount Using a Regular Expression
# We're assuming that the total amount is usually found with the keyword "Total" or "Amount"
amount_pattern = r'(?i)(total|amount)\s*[:\s]*([\d,.]+)'  # Case-insensitive search for "Total" or "Amount"
matches = re.findall(amount_pattern, text)
if matches:
    total_amount = matches[0][1]  # Extract the first match
    print("Detected Total Amount: ", total_amount)
else:
    print("Could not find the total amount in the invoice.")

Could not find the total amount in the invoice.


In [ ]:
# Step 6: Define Categories and Train a Simple Classifier Based on Extracted Text (Text Classification)
# Example categories for invoice classification (you can extend this as needed)
categories = {
    'groceries': ['grocery', 'milk', 'bread', 'food', 'vegetables'],
    'electronics': ['electronics', 'laptop', 'phone', 'charger'],
    'services': ['service', 'repair', 'maintenance'],
}

# Prepare training data (text and their respective categories)
training_data = [
    ('Grocery store invoice with items like milk and bread', 'groceries'),
    ('Invoice for electronic items such as phone and charger', 'electronics'),
    ('Service invoice for car repair and maintenance', 'services'),
]

texts, labels = zip(*training_data)


In [ ]:
# Step 7: Vectorize the Text Data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(texts)


In [ ]:
# Step 8: Train a Simple Naive Bayes Classifier
classifier = MultinomialNB()
classifier.fit(X_train, labels)

MultinomialNB()

In [ ]:
# Step 9: Classify the Uploaded Invoice Text
# Vectorize the extracted text from the invoice
X_test = vectorizer.transform([text])
predicted_category = classifier.predict(X_test)[0]
print("Predicted Invoice Category: ", predicted_category)

Predicted Invoice Category:  services


In [ ]:
# Step 10: Export the Results to CSV
# Store the extracted information in a dictionary
invoice_data = {
    'Invoice File': [image_path],
    'Extracted Amount': [total_amount if matches else 'Not Found'],
    'Predicted Category': [predicted_category],
}


In [ ]:
# Convert to a DataFrame and export to CSV
df = pd.DataFrame(invoice_data)
df.to_csv('invoice_data.csv', index=False)

# Download the CSV
from google.colab import files
files.download('invoice_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>